In [ ]:
!pip install -q google_play_scraper # Scraping reviews
!pip install -q transformers # Sentiment and Summarization
!pip install -q plotly-express # Data Visualization
!pip install pyyaml==5.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import pandas as pd
import numpy as np
from google_play_scraper import app, Sort, reviews_all
import plotly.express as px
from transformers import pipeline
from sklearn.metrics import accuracy_score


In [ ]:
# Scraping reviews for the app "cris.railmadad" from Google Play Store
rm_project = reviews_all('cris.railmadad', sleep_milliseconds=0, lang='en', country='IN', sort=Sort.NEWEST)

# Normalizing the reviews into a DataFrame
df = pd.json_normalize(rm_project)

# Display the first few rows of the dataframe to inspect
df.head()


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a23f629d-c91e-4773-9e73-0048d301f0f2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,excellent 👍,5,0,3.0.16,2025-08-10 15:17:02,None,None,3.0.16
1,701bf1cc-4437-411e-b668-72f88ed6865f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"even after entering the correct PNR number, th...",1,0,3.0.16,2025-08-10 10:47:11,None,None,3.0.16
2,50916aa9-7eb8-4eb9-a8ff-137f30dadb53,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,not working properly,1,0,3.0.16,2025-08-10 09:44:12,None,None,3.0.16
3,5cbdee00-ae13-4615-b5aa-0ed22eb876a8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I am trying so much to login but not get,1,0,None,2025-08-10 03:43:27,None,None,None
4,bc942c4d-00c2-4bbd-8fa1-5b3d614ecb11,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,app login nhi hota jab need hoti h,1,0,3.0.16,2025-08-07 07:11:32,None,None,3.0.16


In [ ]:
# Load the sentiment analysis pipeline for positive/negative sentiment
sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
df['content'] = df['content'].astype('str')

In [ ]:
# Load the BART model for summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to summarize long reviews
def summarize_review(review, max_length=512):
    if len(review.split()) > max_length:
        summary = summarizer(review, max_length=150, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    return review

# Apply summarization to long reviews
df['summarized_content'] = df['content'].apply(lambda x: summarize_review(x))

# Display the first few rows with summarized content
df.head()


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,summarized_content
0,a23f629d-c91e-4773-9e73-0048d301f0f2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,excellent 👍,5,0,3.0.16,2025-08-10 15:17:02,None,None,3.0.16,excellent 👍
1,701bf1cc-4437-411e-b668-72f88ed6865f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"even after entering the correct PNR number, th...",1,0,3.0.16,2025-08-10 10:47:11,None,None,3.0.16,"even after entering the correct PNR number, th..."
2,50916aa9-7eb8-4eb9-a8ff-137f30dadb53,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,not working properly,1,0,3.0.16,2025-08-10 09:44:12,None,None,3.0.16,not working properly
3,5cbdee00-ae13-4615-b5aa-0ed22eb876a8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I am trying so much to login but not get,1,0,None,2025-08-10 03:43:27,None,None,None,I am trying so much to login but not get
4,bc942c4d-00c2-4bbd-8fa1-5b3d614ecb11,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,app login nhi hota jab need hoti h,1,0,3.0.16,2025-08-07 07:11:32,None,None,3.0.16,app login nhi hota jab need hoti h


In [ ]:
# Mapping star ratings (1-3 -> positive, 4-5 -> negative) for ground truth sentiment
def map_star_to_sentiment(star_rating):
    if star_rating in [1, 2]:
        return 'NEGATIVE'   # Consider 1-3 stars as positive
    elif star_rating in [4, 5]:
        return 'POSITIVE'   # 4-5 stars as negative
    else:
      return 'NEUTRAL'      # Optional, for cases with no matching star

# Apply mapping to create ground truth sentiment (based on the original star ratings)
df['ground_truth_sentiment'] = df['score'].apply(map_star_to_sentiment)

# Display the DataFrame with ground truth sentiment
df.head()


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,summarized_content,ground_truth_sentiment
0,a23f629d-c91e-4773-9e73-0048d301f0f2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,excellent 👍,5,0,3.0.16,2025-08-10 15:17:02,None,None,3.0.16,excellent 👍,POSITIVE
1,701bf1cc-4437-411e-b668-72f88ed6865f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"even after entering the correct PNR number, th...",1,0,3.0.16,2025-08-10 10:47:11,None,None,3.0.16,"even after entering the correct PNR number, th...",NEGATIVE
2,50916aa9-7eb8-4eb9-a8ff-137f30dadb53,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,not working properly,1,0,3.0.16,2025-08-10 09:44:12,None,None,3.0.16,not working properly,NEGATIVE
3,5cbdee00-ae13-4615-b5aa-0ed22eb876a8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I am trying so much to login but not get,1,0,None,2025-08-10 03:43:27,None,None,None,I am trying so much to login but not get,NEGATIVE
4,bc942c4d-00c2-4bbd-8fa1-5b3d614ecb11,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,app login nhi hota jab need hoti h,1,0,3.0.16,2025-08-07 07:11:32,None,None,3.0.16,app login nhi hota jab need hoti h,NEGATIVE


In [ ]:
# Apply sentiment analysis using the model
df['result'] = df['summarized_content'].apply(lambda x: sentiment_analysis(x)[0])

# Extract the predicted sentiment (label) and score (confidence) into separate columns
df['sentiment'] = df['result'].apply(lambda x: x['label'])
df['score'] = df['result'].apply(lambda x: x['score'])

# Display the DataFrame with predicted sentiment and sentiment score
df.head()


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,summarized_content,ground_truth_sentiment,result,sentiment
0,a23f629d-c91e-4773-9e73-0048d301f0f2,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,excellent 👍,0.999846,0,3.0.16,2025-08-10 15:17:02,None,None,3.0.16,excellent 👍,POSITIVE,"{'label': 'POSITIVE', 'score': 0.9998458623886...",POSITIVE
1,701bf1cc-4437-411e-b668-72f88ed6865f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"even after entering the correct PNR number, th...",0.595262,0,3.0.16,2025-08-10 10:47:11,None,None,3.0.16,"even after entering the correct PNR number, th...",NEGATIVE,"{'label': 'POSITIVE', 'score': 0.5952618718147...",POSITIVE
2,50916aa9-7eb8-4eb9-a8ff-137f30dadb53,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,not working properly,0.999785,0,3.0.16,2025-08-10 09:44:12,None,None,3.0.16,not working properly,NEGATIVE,"{'label': 'NEGATIVE', 'score': 0.9997847676277...",NEGATIVE
3,5cbdee00-ae13-4615-b5aa-0ed22eb876a8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I am trying so much to login but not get,0.998916,0,None,2025-08-10 03:43:27,None,None,None,I am trying so much to login but not get,NEGATIVE,"{'label': 'NEGATIVE', 'score': 0.9989162683486...",NEGATIVE
4,bc942c4d-00c2-4bbd-8fa1-5b3d614ecb11,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,app login nhi hota jab need hoti h,0.994708,0,3.0.16,2025-08-07 07:11:32,None,None,3.0.16,app login nhi hota jab need hoti h,NEGATIVE,"{'label': 'NEGATIVE', 'score': 0.9947084188461...",NEGATIVE


In [ ]:
# Display the distribution of ground truth sentiments
print(df['ground_truth_sentiment'].value_counts())

# Filter out rows where ground truth sentiment is 'NEUTRAL' (if you want to focus on positive/negative)
df_filtered = df[df['ground_truth_sentiment'] != 'NEUTRAL']

# Check how many rows are left after filtering neutral sentiments
print(f"Rows after filtering for 'NEUTRAL' sentiment: {len(df_filtered)}")

# Calculate accuracy by comparing ground truth sentiment with predicted sentiment
accuracy = accuracy_score(df_filtered['ground_truth_sentiment'], df_filtered['sentiment'])

# Display accuracy as percentage
accuracy_percentage = accuracy * 100
print(f"Accuracy: {accuracy_percentage:.2f}%")


ground_truth_sentiment
NEGATIVE    2008
POSITIVE    1080
NEUTRAL       64
Name: count, dtype: int64
Rows after filtering for 'NEUTRAL' sentiment: 3088
Accuracy: 91.48%


In [ ]:
fig = px.histogram(df, x='sentiment', color='sentiment', text_auto=True)

fig.show()


In [ ]:
def interactive_prediction():
    while True:
        # Input a review text from the user
        user_input = input("Enter a review to predict its sentiment (or type 'exit' to quit): ")

        # Check if user wants to exit
        if user_input.lower() == 'exit':
            print("Exiting interactive prediction.")
            break

        # Summarize the input if it's too long
        summarized_input = summarize_review(user_input)

        # Predict sentiment using the model
        prediction = sentiment_analysis(summarized_input)

        # Output the sentiment prediction
        print(f"Predicted Sentiment: {prediction[0]['label']}")

# Call the interactive prediction function
interactive_prediction()

Enter a review to predict its sentiment (or type 'exit' to quit): hi there it is good
Predicted Sentiment: POSITIVE
Enter a review to predict its sentiment (or type 'exit' to quit): not good
Predicted Sentiment: NEGATIVE
Enter a review to predict its sentiment (or type 'exit' to quit): very bad
Predicted Sentiment: NEGATIVE
Enter a review to predict its sentiment (or type 'exit' to quit): ok ok
Predicted Sentiment: POSITIVE
Enter a review to predict its sentiment (or type 'exit' to quit): quit
Predicted Sentiment: NEGATIVE
Enter a review to predict its sentiment (or type 'exit' to quit): q
Predicted Sentiment: POSITIVE
Enter a review to predict its sentiment (or type 'exit' to quit): exi
Predicted Sentiment: NEGATIVE
Enter a review to predict its sentiment (or type 'exit' to quit): exit
Exiting interactive prediction.
